In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1500)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [2]:
# 数据集
# 数据集由 Cleveland Clinic Foundation 提供的几百行心脏病数据构成。每一行代表一个病人，每一列描述一个属性，数据共14列，最后一列为是否患病。我们将使用这些信息预测一个病人是否有心脏病。这是一个典型的二分分类问题。

# 列	描述	特征类型	数据类型
# Age	年龄	数值	integer
# Sex	性别(1男性； 0女性)	类别	integer
# …	…	…	…
# Thal	3 = normal; 6 = fixed defect; 7 = reversable defect	类别	string
# Target	是否感染，(1是；0否)	分类	integer

In [3]:

import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
print(tf.__version__)

2.0.0-beta1


In [4]:
# 使用 Pandas 读取数据
dataframe = pd.read_csv('heart.csv')
dataframe.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [5]:
# 分割训练集、验证集和测试集
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples') # 193
print(len(val), 'validation examples') # 49
# print(len(test), 'test examples') # 61

193 train examples
49 validation examples


In [6]:
# 创建 input pipeline
# 使用 tf.data ，我们可以使用特征工程(feature columns)将 Pandas DataFrame 中的列映射为特征值(features)。
# 如果是一个非常大的 CSV 文件，不能直接放在内存中，
# 就必须直接使用 tf.data 从磁盘中直接读取数据了。

In [7]:
# 帮助函数，返回 tf.data 数据集。
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [8]:
batch_size = 5 # 为了方面展示下面的示例，batch暂时设置为5。
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [9]:
# 理解 input pipeline
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of ages:', feature_batch['age'])
  print('A batch of targets:', label_batch )

Every feature: ['age', 'exang', 'thalach', 'thal', 'fbs', 'cp', 'chol', 'sex', 'restecg', 'trestbps', 'oldpeak', 'ca', 'slope']
A batch of ages: tf.Tensor([62 64 56 68 59], shape=(5,), dtype=int32)
A batch of targets: tf.Tensor([1 0 0 1 0], shape=(5,), dtype=int32)


In [10]:
# 特征列示例
# TensorFlow 提供了很多种类型的特征列(feature column)，
# 接下来给几个例子看一看每一列的值是怎么被转换的
example_batch = next(iter(train_ds))[0]

# 帮助函数：创建一个特征列，并转换。
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

In [11]:
# 1) Numeric column
# 特征列的输出是模型的输入，Numeric columns 是最简单的类型，
# 数值本身代表某个特征真实的值，
# 因此转换后，值不发生改变。
age = feature_column.numeric_column("age")
demo(age)

[[62.]
 [64.]
 [56.]
 [68.]
 [59.]]


In [12]:
# 2) Bucketized columns
# 有时候，并不想直接将数值传给模型，而是希望基于数值的范围离散成几个种类。
# 比如人的年龄，0-10归为一类，用0表示；11-20归为一类，用1表示。
# 我们可以用 bucketized column 将年龄划分到不同的 bucket 中。
# 用中文比喻，就好像提供了不同的桶，在某一范围内的扔进A桶，另一范围的数据扔进B桶，以此类推。
# 下面的例子使用独热编码来表示不同的 bucket
age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
demo(age_buckets)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]


In [13]:
# 3) Categorical columns
# 在这个数据集中，thal列使用字符串表示(e.g. ‘fixed’, ‘normal’, ‘reversible’)。
# 字符串不能直接传给模型。所以我们要先将字符串映射为数值。
# 可以使用categorical_column_with_vocabulary_list 和 categorical_column_with_vocabulary_file 来转换，
# 前者接受一个列表作为输入，后者可以传入一个文件。

# 可以看到，最终的输出向量也是独热编码，和 Bucketized column 相似。

thal = feature_column.categorical_column_with_vocabulary_list(
      'thal', ['fixed', 'normal', 'reversible'])

thal_one_hot = feature_column.indicator_column(thal)
demo(thal_one_hot)

W0715 01:29:47.730547 139686601557760 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/feature_column/feature_column_v2.py:2655: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0715 01:29:47.740763 139686601557760 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/feature_column/feature_column_v2.py:4215: IndicatorColumn._variable_shape (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed in a future version.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
W0715 01:29:47.742153 139686601557760 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/feature_column/feature_column_v2.py:4270: VocabularyListCateg

[[0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]


In [14]:
# 4) Embedding column
# 假设某一列有上千种类别，用独热编码来表示就不太合适了。
# 这时候，可以使用 embedding column。embedding column 可以压缩维度，因此向量中的值不再只由0或1组成，可以包含任何数字。

# 在有很多种类别时使用 embedding column 是最合适的。
# 接下来只是一个示例，不管输入有多少种可能性，最终的输出向量定长为8。
# embedding column的输入是categorical column
thal_embedding = feature_column.embedding_column(thal, dimension=8)
demo(thal_embedding)

[[-0.03135112  0.17630544  0.4094072   0.02513588 -0.2360947  -0.30939803
  -0.06512725  0.38364524]
 [-0.5045371  -0.50747824 -0.597579   -0.3737529  -0.44021395 -0.08897981
  -0.07711721  0.2576472 ]
 [-0.03135112  0.17630544  0.4094072   0.02513588 -0.2360947  -0.30939803
  -0.06512725  0.38364524]
 [-0.5045371  -0.50747824 -0.597579   -0.3737529  -0.44021395 -0.08897981
  -0.07711721  0.2576472 ]
 [-0.03135112  0.17630544  0.4094072   0.02513588 -0.2360947  -0.30939803
  -0.06512725  0.38364524]]


In [15]:
# 5) Hashed feature columns
# 另一种表示类别很多的 categorical column 的方式是使用 categorical_column_with_hash_bucket。
# 这个特征列会计算输入的哈希值，然后根据哈希值对字符串进行编码。哈希桶(bucket)个数即参数hash_bucket_size。
# 哈希桶(hash_buckets)的个数应明显小于实际的类别个数，以节省空间。

# 注意：哈希的一大副作用是可能存在冲突，不同的字符串可能映射到相同的哈希桶中。
# 不过，在某些数据集，这个方式还是非常有效的。
thal_hashed = feature_column.categorical_column_with_hash_bucket(
      'thal', hash_bucket_size=1000)
demo(feature_column.indicator_column(thal_hashed))

W0715 01:29:48.625992 139686601557760 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/feature_column/feature_column_v2.py:4270: HashedCategoricalColumn._num_buckets (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed in a future version.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [16]:
# 6) Crossed feature columns
# 将几个特征组合成一个特征，即 feature crosses，模型可以对每一个特征组合学习独立的权重。
# 接下来，我们将组合 age 和 thal 列创建一个新的特征。
# 注意：crossed_column不会创建所有可能的组合，因为组合可能性会非常多。
# 背后是通过hashed_column处理的，可以设置哈希桶的大小。
crossed_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
demo(feature_column.indicator_column(crossed_feature))

W0715 01:29:48.968266 139686601557760 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/feature_column/feature_column_v2.py:4270: CrossedColumn._num_buckets (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed in a future version.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [17]:
# 选择需要使用的列
# 为了训练出准确率高的模型，大数据集、选取有意义的列、数据的展示方式都是非常重要的。

# 接下来的示例，我们随机选取一些列来训练。
feature_columns = []

# numeric cols
for header in ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'slope', 'ca']:
  feature_columns.append(feature_column.numeric_column(header))

# bucketized cols
age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
feature_columns.append(age_buckets)

# indicator cols
thal = feature_column.categorical_column_with_vocabulary_list(
      'thal', ['fixed', 'normal', 'reversible'])
thal_one_hot = feature_column.indicator_column(thal)
feature_columns.append(thal_one_hot)

# embedding cols
thal_embedding = feature_column.embedding_column(thal, dimension=8)
feature_columns.append(thal_embedding)

# crossed cols
crossed_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
crossed_feature = feature_column.indicator_column(crossed_feature)
feature_columns.append(crossed_feature)

In [18]:
# 创建特征层
# 我已经定义好了特征列，接下来使用 DenseFeatures 层将特征列传入到模型中。
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)



 

In [19]:
# 之前 batch 大小设置为5，是为了方便示例。接下来batch设置为32，创建新的 input pipeline。
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)


In [20]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'],
              run_eagerly=True)

model.fit(train_ds,
          validation_data=val_ds,
          epochs=5)

Epoch 1/5
7/7 [==============================] - 2s 265ms/step - loss: 4.1678 - accuracy: 0.3635 - val_loss: 3.4261 - val_accuracy: 0.6531
Epoch 2/5
7/7 [==============================] - 1s 82ms/step - loss: 1.8130 - accuracy: 0.7679 - val_loss: 0.7299 - val_accuracy: 0.5714
Epoch 3/5
7/7 [==============================] - 1s 79ms/step - loss: 0.5904 - accuracy: 0.6891 - val_loss: 1.1660 - val_accuracy: 0.6735
Epoch 4/5
7/7 [==============================] - 1s 84ms/step - loss: 0.7328 - accuracy: 0.7757 - val_loss: 0.6325 - val_accuracy: 0.6531
Epoch 5/5
7/7 [==============================] - 1s 80ms/step - loss: 0.4430 - accuracy: 0.7924 - val_loss: 0.7651 - val_accuracy: 0.6735


In [21]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

2/2 [==============================] - 0s 42ms/step - loss: 0.4981 - accuracy: 0.7705
Accuracy 0.7704918


In [ ]:
# 如果你使用深度学习模型，以及更大、更复杂的数据集，准确率会更高。
# 一般来说，像这样的小数据集，建议使用决策树或者随机森林。
# 这个教程的主要目的不是训练一个准确率高的模型，
# 而是作一个示例：TensorFlow 如何处理结构化的数据。